In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import reward as rw
import reward.utils as U

/home/quazar/anaconda3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
GAMMA = 0.99

In [3]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

In [4]:
env = rw.env.GymEnv("InvertedPendulum-v2")
runner = rw.runner.SingleRunner(env)
batcher = rw.batcher.RolloutBatcher(runner=runner, batch_size=512)

Choosing the latest nvidia driver: /usr/lib/nvidia-390, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-390']
Choosing the latest nvidia driver: /usr/lib/nvidia-390, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-390']
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [5]:
class PolicyNN(nn.Module):
    def __init__(self, num_inputs, num_outputs, activation=nn.Tanh):
        super().__init__()
        self.activation = activation()
        
        self.hidden = nn.Linear(num_inputs, 64)
        self.mean = nn.Linear(64, num_outputs)
        self.log_std = nn.Parameter(torch.zeros(1, num_outputs))
        
    def forward(self, x):
        x = self.activation(self.hidden(x))
        mean = self.mean(x)
        log_std = self.log_std.expand_as(mean)
        return mean, log_std        

In [6]:
class ValueNN(nn.Module):
    def __init__(self, num_inputs, activation=nn.Tanh):
        super().__init__()
        self.activation = activation()
        
        self.hidden = nn.Linear(num_inputs, 64)
        self.out = nn.Linear(64, 1)
        
    def forward(self, x):
        x = self.activation(self.hidden(x))
        return self.out(x)

In [7]:
class GaussianPolicy(rw.policy.BasePolicy):
    def create_dist(self, state):
        mean, log_std = self.nn(state)
        return rw.distributions.Normal(loc=mean, scale=log_std.exp())
    
    def get_action(self, state, step):
        dist = self.create_dist(state)
        return U.to_np(dist.sample())

In [8]:
p_nn = PolicyNN(batcher.state_space.shape[0], batcher.action_space.shape[0]).to(device)
v_nn = ValueNN(batcher.state_space.shape[0]).to(device)
policy = GaussianPolicy(p_nn)

p_opt = torch.optim.Adam(p_nn.parameters(), lr=1e-3)
v_opt = torch.optim.Adam(v_nn.parameters(), lr=1e-3)

In [9]:
logger = U.Logger('logs/inverted_pendulum/v0-0')
last_logged_step = 0

Writing logs to: logs/inverted_pendulum/v0-0


In [10]:
for batch in batcher.get_batches(max_steps=5e5, act_fn=policy.get_action):
    batch = batch.to_tensor()
    # Calculate state value
    s = U.join_first_dims(batch.s, 2)
    v_t = v_nn(s)
    # Calculate return
    ret = U.estimators.discounted_sum_rs(
        rs=batch.r,
        dones=batch.done,
        gamma=GAMMA,
        v_t_last=v_t[-1]
    ).detach()
    batch = batch.concat_batch()
    # Calculate advantage    
    adv = (ret - v_t).detach()
    
    # Calculate policy loss
    dist = policy.create_dist(batch.s)
    log_prob = dist.log_prob(batch.action)
    assert ret.shape == log_prob.shape
    p_loss = -(adv * log_prob).mean()
    
    # Calculate value loss
    v_loss = F.mse_loss(v_t, ret)
    
    # Optimize
    p_opt.zero_grad()
    p_loss.backward()
    p_opt.step()
    
    v_opt.zero_grad()
    v_loss.backward()
    v_opt.step()
    
    # Write logs    
    if batcher.num_steps > last_logged_step:
        last_logged_step = batcher.num_steps + 10000
        batcher.write_logs(logger)
        logger.add_log('policy/loss', p_loss)
        logger.add_log('v/loss', v_loss)
        logger.log(step=batcher.num_steps)      

HBox(children=(IntProgress(value=0, layout=Layout(flex='2'), max=500000), HTML(value='')), layout=Layout(display='inline-flex', flex_flow='row wrap', width='100%'))


                          Step 512                          
--------------------------------------------------------------
Env/Reward/Episode (New)                               |  7.52
Env/Length/Episode (New)                               |  7.52
Env/Reward/Episode (Last 50)                           |  7.64
Env/Length/Episode (Last 50)                           |  7.64
policy/loss                                            |  6.70
v/loss                                                 | 39.49
--------------------------------------------------------------

                         Step 10752                         
--------------------------------------------------------------
Env/Reward/Episode (New)                               |  8.84
Env/Length/Episode (New)                               |  8.84
Env/Reward/Episode (Last 50)                           | 10.78
Env/Length/Episode (Last 50)                           | 10.78
policy/loss                                            | 


                        Step 154112                        
--------------------------------------------------------------
Env/Reward/Episode (New)                             |  387.23
Env/Length/Episode (New)                             |  387.23
Env/Reward/Episode (Last 50)                         |  411.00
Env/Length/Episode (Last 50)                         |  411.00
policy/loss                                          |   37.77
v/loss                                               | 1481.58
--------------------------------------------------------------

                        Step 164352                        
--------------------------------------------------------------
Env/Reward/Episode (New)                             |  403.44
Env/Length/Episode (New)                             |  403.44
Env/Reward/Episode (Last 50)                         |  400.56
Env/Length/Episode (Last 50)                         |  400.56
policy/loss                                          |   60


                        Step 307712                        
--------------------------------------------------------------
Env/Reward/Episode (New)                             |  955.64
Env/Length/Episode (New)                             |  955.64
Env/Reward/Episode (Last 50)                         |  802.34
Env/Length/Episode (Last 50)                         |  802.34
policy/loss                                          |   34.10
v/loss                                               | 1216.69
--------------------------------------------------------------

                        Step 317952                        
--------------------------------------------------------------
Env/Reward/Episode (New)                             |  506.85
Env/Length/Episode (New)                             |  506.85
Env/Reward/Episode (Last 50)                         |  702.44
Env/Length/Episode (Last 50)                         |  702.44
policy/loss                                          |   33


                        Step 461312                        
--------------------------------------------------------------
Env/Reward/Episode (New)                             | 1000.00
Env/Length/Episode (New)                             | 1000.00
Env/Reward/Episode (Last 50)                         |  961.18
Env/Length/Episode (Last 50)                         |  961.18
policy/loss                                          |   34.96
v/loss                                               |  905.35
--------------------------------------------------------------

                        Step 471552                        
--------------------------------------------------------------
Env/Reward/Episode (New)                             | 1000.00
Env/Length/Episode (New)                             | 1000.00
Env/Reward/Episode (Last 50)                         |  985.60
Env/Length/Episode (Last 50)                         |  985.60
policy/loss                                          |   16